In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder, LabelEncoder

In [2]:
data = pd.read_csv('CleanData.csv')

In [3]:
# data['Order Date'] = pd.to_datetime(data['Order Date'])

# groupedData = data.groupby(['Order Date','Segment',
#                              'Country', 'Sub-Category',
#                                'PromotionFlag']).agg({
#                                     'Sales': 'sum'
#                                }).reset_index()

# groupedData

In [16]:
filtered = pd.read_csv('Grouped.csv')
filtered.drop(['Unnamed: 0'],inplace=True, axis=1)
filtered

,Order Date,Segment,Country,Product ID,PromotionFlag,Sales
0,2012-01-01,Consumer,Algeria,OFF-ST-6261,0,408.300
1,2012-01-01,Consumer,Australia,FUR-FU-4075,1,113.670
2,2012-01-01,Consumer,Australia,OFF-PA-3990,1,55.242
3,2012-01-01,Consumer,Australia,OFF-SU-3002,1,120.366
4,2012-01-01,Consumer,Hungary,OFF-ST-6230,0,66.120
...,...,...,...,...,...,...
51221,2015-12-31,Home Office,Nicaragua,OFF-ST-6024,0,15.000
51222,2015-12-31,Home Office,Spain,TEC-CO-4568,0,530.220
51223,2015-12-31,Home Office,Tanzania,OFF-LA-4542,0,49.500
51224,2015-12-31,Home Office,Trinidad and Tobago,OFF-AP-3858,0,135.480


In [6]:
prod_info = data[['Product ID', 'Sub-Category', 'Category']].drop_duplicates()

In [7]:
merged = filtered.merge(prod_info , on='Product ID',how='left')

merged

,Order Date,Segment,Country,Product ID,PromotionFlag,Sales,Sub-Category,Category
0,2012-01-01,Consumer,Algeria,OFF-ST-6261,0,408.300,Storage,Office Supplies
1,2012-01-01,Consumer,Australia,FUR-FU-4075,1,113.670,Furnishings,Furniture
2,2012-01-01,Consumer,Australia,OFF-PA-3990,1,55.242,Paper,Office Supplies
3,2012-01-01,Consumer,Australia,OFF-SU-3002,1,120.366,Supplies,Office Supplies
4,2012-01-01,Consumer,Hungary,OFF-ST-6230,0,66.120,Storage,Office Supplies
...,...,...,...,...,...,...,...,...
51221,2015-12-31,Home Office,Nicaragua,OFF-ST-6024,0,15.000,Storage,Office Supplies
51222,2015-12-31,Home Office,Spain,TEC-CO-4568,0,530.220,Copiers,Technology
51223,2015-12-31,Home Office,Tanzania,OFF-LA-4542,0,49.500,Labels,Office Supplies
51224,2015-12-31,Home Office,Trinidad and Tobago,OFF-AP-3858,0,135.480,Appliances,Office Supplies


In [8]:
merged['Order Date'] = pd.to_datetime(merged['Order Date'])

merged['Year'] = merged['Order Date'].dt.year
merged['Month'] = merged['Order Date'].dt.month
merged['Weekday'] = merged['Order Date'].dt.weekday
merged['IsWeekend'] = merged['Weekday'].isin([5, 6])

In [9]:
merged['Season'] = merged['Month'].apply(lambda x: 'Winter' if x in [12, 1, 2] else
                                           'Spring' if x in [3, 4, 5] else
                                           'Summer' if x in [6, 7, 8] else
                                           'Fall')

In [10]:
encoder = OneHotEncoder(sparse = False)

encoded = encoder.fit_transform(merged[['Segment', 'Sub-Category', 'Category', 'Season']])

encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['Segment', 'Sub-Category', 'Category', 'Season']))

data_encoded = pd.concat([merged[['Order Date', 'Year', 'Month', 'Weekday', 'IsWeekend', 'Sales','Country', 'PromotionFlag', 'Product ID']], encoded_df], axis=1)

data_encoded

c:\Users\Farida\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Order Date,Year,Month,Weekday,IsWeekend,Sales,Country,PromotionFlag,Product ID,Segment_Consumer,...,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables,Category_Furniture,Category_Office Supplies,Category_Technology,Season_Fall,Season_Spring,Season_Summer,Season_Winter
0,2012-01-01,2012,1,6,True,408.300,Algeria,0,OFF-ST-6261,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2012-01-01,2012,1,6,True,113.670,Australia,1,FUR-FU-4075,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2012-01-01,2012,1,6,True,55.242,Australia,1,OFF-PA-3990,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,2012-01-01,2012,1,6,True,120.366,Australia,1,OFF-SU-3002,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,2012-01-01,2012,1,6,True,66.120,Hungary,0,OFF-ST-6230,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51221,2015-12-31,2015,12,3,False,15.000,Nicaragua,0,OFF-ST-6024,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
51222,2015-12-31,2015,12,3,False,530.220,Spain,0,TEC-CO-4568,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
51223,2015-12-31,2015,12,3,False,49.500,Tanzania,0,OFF-LA-4542,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
51224,2015-12-31,2015,12,3,False,135.480,Trinidad and Tobago,0,OFF-AP-3858,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [11]:
le_country = LabelEncoder()
le_product_id = LabelEncoder()

data_encoded['Country'] = le_country.fit_transform(data_encoded['Country'])
data_encoded['Product ID'] = le_product_id.fit_transform(data_encoded['Product ID'])

data_encoded

,Order Date,Year,Month,Weekday,IsWeekend,Sales,Country,PromotionFlag,Product ID,Segment_Consumer,...,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables,Category_Furniture,Category_Office Supplies,Category_Technology,Season_Fall,Season_Spring,Season_Summer,Season_Winter
0,2012-01-01,2012,1,6,True,408.300,2,0,2732,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2012-01-01,2012,1,6,True,113.670,6,1,513,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2012-01-01,2012,1,6,True,55.242,6,1,2151,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,2012-01-01,2012,1,6,True,120.366,6,1,2801,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,2012-01-01,2012,1,6,True,66.120,65,0,2722,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51221,2015-12-31,2015,12,3,False,15.000,105,0,2688,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
51222,2015-12-31,2015,12,3,False,530.220,134,0,3230,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
51223,2015-12-31,2015,12,3,False,49.500,144,0,2035,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
51224,2015-12-31,2015,12,3,False,135.480,148,0,903,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [12]:
data_encoded.columns

Index(['Order Date', 'Year', 'Month', 'Weekday', 'IsWeekend', 'Sales',
       'Country', 'PromotionFlag', 'Product ID', 'Segment_Consumer',
       'Segment_Corporate', 'Segment_Home Office', 'Sub-Category_Accessories',
       'Sub-Category_Appliances', 'Sub-Category_Art', 'Sub-Category_Binders',
       'Sub-Category_Bookcases', 'Sub-Category_Chairs', 'Sub-Category_Copiers',
       'Sub-Category_Envelopes', 'Sub-Category_Fasteners',
       'Sub-Category_Furnishings', 'Sub-Category_Labels',
       'Sub-Category_Machines', 'Sub-Category_Paper', 'Sub-Category_Phones',
       'Sub-Category_Storage', 'Sub-Category_Supplies', 'Sub-Category_Tables',
       'Category_Furniture', 'Category_Office Supplies', 'Category_Technology',
       'Season_Fall', 'Season_Spring', 'Season_Summer', 'Season_Winter'],
      dtype='object')

In [17]:
data_encoded.to_csv("EncodedData.csv",index=False)